In [1]:
import numpy as np
import pandas as pd
import ast

from datetime import datetime
import time

from list_of_words import *
from keyword_finder import find_words, doctors_identifier,trash_identifier
from drop_user_id import drop_user_ids

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Done 1. How many post per user
# Done 2. How many time the same post occured
# Done 3. How many users posted the same post (for different posts)
# Done 4. How many time the user posted the same post 
# 5. If the time difference of the same posts done by 1 user is < threshold. join all infos (likes, comments...) of the posts

In [4]:
%%time
df = pd.read_csv('user_info_filtered_new_20July2018.csv')#('user_info_filtered.csv') user_info_filtered_posts.csv
#df.drop(df.columns[0], axis=1,inplace=True)

Wall time: 47.9 s


In [5]:
df.shape,df.user_id.nunique()

(249510, 30)

In [7]:
#posts with less then 200 words
df['short_post']=df.post.apply(lambda x: x if len(x.split(' '))<200 else 'Too Long') 
#taking only short posts to work with
df_short = df[df.short_post!='Too Long']

In [45]:
df_short.head(1)

,user_id,date,post,Name_Surname,activities,bdate,city,comments,comments_count,education_status,faculty_name,graduation_year,interests,key_word,likes_count,platform,post_id,reposts_count,sex,site,university,users_liked,users_reposted,work_place,work_type,doctor,oculist,pediator,client,trash,post_count_per_user,same_post_count,users_count_per_post,same_post_count_per_user,post_kosmet_keyword_work_set,post_kosmet_keyword_work_count,post_kosmet_keyword_act_set,post_kosmet_keyword_act_count,post_kosmet_keyword_web_set,post_kosmet_keyword_web_count,post_isanimals,post_kosmet_sum
0,636,2014-01-01 22:52:18,"Добро пожаловать, гости дорогие.",Alexander_Belyaev,NaN,1.5.1989,Pushkin,"{'count': 4, 'items': [{'id': 28137, 'from_id'...",4,NaN,NaN,0.0,NaN,красный глаз помогите,27,vk,28136.0,16,Male,http://vk.com/dekadence,NaN,"{'count': 27, 'items': [542303, 776496, 103315...","{'count': 16, 'items': [-45399178, -38699240, ...",Гражданин Пушкин,work,0,0,0,0,0,1,1,1,1,[],0,[],0,[],0,0,0


## Idea N1 Start

In [10]:
#counting number of user and reseting index
df_user_count=df_short.user_id.value_counts().reset_index()

#renaming columns names
df_user_count.rename(columns={"user_id": "post_count_per_user","index": "user_id", },inplace=True)

#merging our main df_short and df_user_count dataframes
df_short=pd.merge(df_short,df_user_count,how = 'left', on='user_id')

## Idea N1 End

## Idea N2 Start

In [11]:
#counting number of posts and reseting index
df_post_count=df_short.post.value_counts().reset_index()

#renaming columns names
df_post_count.rename(columns={"post": "same_post_count","index": "post", },inplace=True)

#merging our main df_short and df_post_count dataframes
df_short=pd.merge(df_short,df_post_count,how = 'left', on='post')

## Idea N2 End

## Idea N3 Start

In [12]:
#groupping by post and taking the number of users with that post
df_same_post_per_user = df_short.groupby('post')['user_id'].apply(lambda x :len(set(x)))

#crearing dataframe and resetting index
df_same_post_per_user = pd.DataFrame(df_same_post_per_user).reset_index()

#renaming column name
df_same_post_per_user.rename(columns={"user_id": "users_count_per_post", },inplace=True)

#merging our main df_short and df_same_post_per_user dataframes
df_short=pd.merge(df_short,df_same_post_per_user,how = 'left', on='post')

## Idea N3 Start

## Idea N4 Start

In [13]:
#groupping per user_id and short_post to count how many of the same posts exist per user
df_short_gr = df_short.groupby(['user_id','short_post'])['user_id','short_post'].count()

#renaming columns names and reseting index
df_short_gr.rename(columns={"user_id": "user_count", "short_post": "same_post_count_per_user"},inplace=True) 
df_short_gr.reset_index(inplace = True)

#merging our main df_short and df_short_gr dataframes
df_short=pd.merge(df_short,df_short_gr,how = 'left', on=['user_id','short_post']) # ppu_count --> post per user

#dropping redundant columns
df_short.drop(['short_post','user_count'],axis=1, inplace=True)

## Idea N4 End

In [14]:
pd.options.display.max_columns=50
df_short.head(1)

,user_id,date,post,Name_Surname,activities,bdate,city,comments,comments_count,education_status,faculty_name,graduation_year,interests,key_word,likes_count,platform,post_id,reposts_count,sex,site,university,users_liked,users_reposted,work_place,work_type,doctor,oculist,pediator,client,trash,post_count_per_user,same_post_count,users_count_per_post,same_post_count_per_user
0,636,2014-01-01 22:52:18,"Добро пожаловать, гости дорогие.",Alexander_Belyaev,NaN,1.5.1989,Pushkin,"{'count': 4, 'items': [{'id': 28137, 'from_id'...",4,NaN,NaN,0.0,NaN,красный глаз помогите,27,vk,28136.0,16,Male,http://vk.com/dekadence,NaN,"{'count': 27, 'items': [542303, 776496, 103315...","{'count': 16, 'items': [-45399178, -38699240, ...",Гражданин Пушкин,work,0,0,0,0,0,1,1,1,1


In [16]:
df_short = df_short[df_short.post_count_per_user<80]
df_short = df_short[df_short.same_post_count<20]
df_short = df_short[df_short.users_count_per_post<20]
df_short = df_short[df_short.same_post_count_per_user<15]

In [19]:
%%time
#trash
df_short['post_kosmet_keyword_work_set'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_work,'post','set'),axis=1)
df_short['post_kosmet_keyword_work_count'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_work,'post','count'),axis=1)

Wall time: 22.8 s


In [20]:
%%time
#trash
df_short['post_kosmet_keyword_act_set'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_act,'post','set'),axis=1)
df_short['post_kosmet_keyword_act_count'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_act,'post','count'),axis=1)

Wall time: 23.6 s


In [21]:
%%time
df_short['post_kosmet_keyword_web_set'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_web,'post','set'),axis=1)
df_short['post_kosmet_keyword_web_count'] = df_short.apply(lambda x: find_words(x,kosmet_keyword_web,'post','count'),axis=1)

Wall time: 16.5 s


In [22]:
%%time
#animals
df_short['post_isanimals'] = df_short.apply(lambda x: find_words(x,list_of_animals,'post','exist'),axis=1)

Wall time: 8.81 s


In [26]:
df_short['post_kosmet_sum'] = df_short['post_kosmet_keyword_work_count'] + df_short['post_kosmet_keyword_act_count'] +\
df_short['post_kosmet_keyword_web_count']+df_short['trash']

In [31]:
tr=df_short[df_short['post_kosmet_sum']==0]
tr=tr[~tr.user_id.isin(drop_user_ids)]

In [33]:
tr_nonanimal=tr[tr.post_isanimals==0]

In [46]:
%%time
tr_nonanimal['post_num_keywords'] = tr_nonanimal.apply(lambda x: find_words(x,list_of_keywords_posts,'post','count'),axis=1)
tr_nonanimal['post_set_keywords'] = tr_nonanimal.apply(lambda x: find_words(x,list_of_keywords_posts,'post','set'),axis=1)

c:\users\levon-pc\anaconda3\envs\vk\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Wall time: 18.5 s


c:\users\levon-pc\anaconda3\envs\vk\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
tr.to_csv('cleaned_last.csv')

In [6]:
df.to_csv('user_info_filtered_posts.csv')

In [45]:
"ГРУППА https://vk.com/club.ortolinza26!!! Мы предоставляем нашим пациентам широкий спектр услуг в области диагностики и коррекции зрения. А ТАКЖЕ проводим необходимые исследования на самом высококачественном современном оборудовании.. Все специалисты имеют высокую квалификацию, которую мы продолжаем совершенствовать: регулярно проводятся обучающие тренинги, повышающие уровень знаний и качество обслуживания. У нас всегда можно приобрести самые качественные линзы, оправы, а также сопутствующие товары, аксессуары и средства по уходу. Наша компания сотрудничает с мировыми лидерами на рынке оптики. Мы всегда думаем о своих клиентах. У нас постоянно действуют социальные программы для детей, молодежи и пенсионеров. Акции от нашей компании радуют как постоянных, так и новых клиентов. Также мы периодически проводим конкурсы с ценными призами. Все это мы делаем для того, чтоб вы были уверены, что мы вам всегда рады, вас всегда ждут. Мы продолжаем совершенствовать себя и свои услуги каждую минуту. Все это мы делаем для того, чтобы вы ощутили радость от окружающего мира"

'ГРУППА https://vk.com/club.ortolinza26!!! Мы предоставляем нашим пациентам широкий спектр услуг в области диагностики и коррекции зрения. А ТАКЖЕ проводим необходимые исследования на самом высококачественном современном оборудовании.. Все специалисты имеют высокую квалификацию, которую мы продолжаем совершенствовать: регулярно проводятся обучающие тренинги, повышающие уровень знаний и качество обслуживания. У нас всегда можно приобрести самые качественные линзы, оправы, а также сопутствующие товары, аксессуары и средства по уходу. Наша компания сотрудничает с мировыми лидерами на рынке оптики. Мы всегда думаем о своих клиентах. У нас постоянно действуют социальные программы для детей, молодежи и пенсионеров. Акции от нашей компании радуют как постоянных, так и новых клиентов. Также мы периодически проводим конкурсы с ценными призами. Все это мы делаем для того, чтоб вы были уверены, что мы вам всегда рады, вас всегда ждут. Мы продолжаем совершенствовать себя и свои услуги каждую минут

In [ ]:
БЕСПЛАТНЫЕ  Cсылка  регистрацию  шанс   #оптика  #цветные_линзы #контактные_линзы

In [41]:
tr[tr.user_id==367495578].post.iloc[2]

'➡Тесты для проверки зрения | Хотите - верьте, хотите - нет!<br><br>Всем друзьям и просто хорошим людям - Доброе Утро! ☀<br><br>Если символы на зеленом фоне вам кажутся более четкими, это значит, что у вас, скорее всего, дальнозоркость. Если же наоборот — то у вас начинают проявляться первые признаки близорукости. <br><br>Офтальмологическая консультация - http://zir.ck.ua/on-line-consultation<br><br>#глаза #здоровье_глаз #офтальмолог #вопрос_офтальмологу #зрение #оптика #цветные_линзы #контактные_линзы'

In [157]:
short_df = df[df['post']!='Too Long'].reset_index().drop('index', axis = 1)
short = pd.DataFrame(df_short.key_word.value_counts()/df_short.shape[0])
short.reset_index(inplace=True)
short.rename(columns={"index": "key_word", "key_word": "short_metric"},inplace=True)
main =  pd.DataFrame(df.key_word.value_counts()/df.shape[0])
main.reset_index(inplace=True)
main.rename(columns={"index": "key_word", "key_word": "main_metric"},inplace=True)
shmain = pd.merge(short, main, on = 'key_word')
shmain['diff'] = shmain['short_metric']-shmain['main_metric']

In [171]:
shmain[shmain['diff']>0]

,key_word,short_metric,main_metric,diff
0,#глаза,0.563148,0.185544,0.377604
2,#глазки,0.150444,0.042728,0.107717
3,#зрение,0.022413,0.007278,0.015135
4,#офтальмология,0.011081,0.004974,0.006108
5,#офтальмолог,0.007388,0.002208,0.005179
7,#ячмень,0.003561,0.000994,0.002567
8,#лазернаякоррекциязрения,0.003444,0.000986,0.002458
11,#близорукость,0.002149,0.000826,0.001323
13,#лазернаякоррекция,0.001898,0.000549,0.001349
14,#lasik,0.001737,0.000513,0.001224


In [ ]:
#a = tr.date.apply(lambda x: time.mktime(datetime.strptime(x[:10], '%Y-%m-%d').timetuple()))
#c= b['post_set_keywords'].apply(lambda x : str(x))

In [ ]:
закисает глазик
- косит глазик, убегает глаз
- красный глаз помогите
- гимнастика для глаз, ортоптисты, Бейтс, Норбеков
- убрать нистагм, вылечить нистагм
- сколько операций нужно на косоглазие 
- подскажите верно ли назначили лечение
- какие очки, линзы для детей 
- ЧАЗН, паралитическое косоглазие, лечение и диагностика ЧАЗН
- сколько стоит операция на нистагм/косоглазие/лазерная коррекция, оперируете
расходящееся/сходящееся косоглазие, операция под наркозом?
- какие именно аппараты будут делать на аппаратном лечении, стоимость аппаратного
лечения,
- можно ли нам капать, подскажите капли
- что такое очки амблиз
- как носить окклюдер/заклейку
- бинокулярное зрение, 3D-формат 
- как лечить близорукость
- что лучше носить очки или линзы?
- до 18 лет лазерную коррекцию делать нельзя
- склеропластика, нужно ли делать
- очки вызывают привыкание, стоит их носить?
- с какого возраста проводится аппаратное лечение
- есть ли шанс снять очки
- прогрессивные очки/бифокальные очки что такое?